# Section 1: データ探索 + チャーンラベル作成

## 🎯 このセクションの目標
- 生データ（顧客・注文）の構造を理解する
- 顧客の注文パターンを分析する
- **チャーンの定義を決め、ラベルを作成する**

## 📊 シナリオ

> あなたは小売会社「Snow Retail」のデータサイエンティストです。
> 
> **2024年前半**のデータを分析し、**2024年後半**に離反する顧客を予測したいと考えています。

---
## 1.1 環境設定

In [ ]:
# 必要なライブラリのインポート
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd

# アクティブセッションの取得
session = get_active_session()

# コンテキストの確認
print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")
print(f"Warehouse: {session.get_current_warehouse()}")

In [ ]:
# 使用するデータベース・スキーマを設定
session.use_database("MLOPS_HOL_DB")
session.use_schema("PREP_DATA")
session.use_warehouse("MLOPS_HOL_SQL_WH")

print("✅ 環境設定完了！")

---
## 1.2 データの確認

GitHubリポジトリからロードした2つのテーブルを確認します。

| テーブル | 説明 |
|----------|------|
| CUSTOMERS | 顧客マスタ（3,000件） |
| ORDERS | 注文履歴（約20,000件） |

In [ ]:
# 顧客データの確認
customers = session.table("CUSTOMERS")
print(f"顧客数: {customers.count():,}")
print(f"\nカラム: {customers.columns}")
customers.show(5)

In [ ]:
# 注文データの確認
orders = session.table("ORDERS")
print(f"注文数: {orders.count():,}")
print(f"\nカラム: {orders.columns}")
orders.show(5)

In [ ]:
# 顧客セグメント別分布
print("=== 顧客セグメント別分布 ===")
customers.group_by("SEGMENT").count().order_by("COUNT", ascending=False).show()

In [ ]:
# 地域別分布
print("=== 地域別分布 ===")
customers.group_by("REGION").count().order_by("COUNT", ascending=False).show()

---
## 1.3 注文データの分析

In [ ]:
# 注文日付の範囲を確認
date_range = orders.agg(
    F.min("ORDER_DATE").alias("MIN_DATE"),
    F.max("ORDER_DATE").alias("MAX_DATE")
).collect()[0]

print("=== 注文日付の範囲 ===")
print(f"  最小: {date_range['MIN_DATE']}")
print(f"  最大: {date_range['MAX_DATE']}")

In [ ]:
# 注文ステータス別件数
print("=== 注文ステータス別件数 ===")
orders.group_by("STATUS").count().order_by("COUNT", ascending=False).show()

In [ ]:
# 月別注文件数（2024年）
print("=== 2024年 月別注文件数 ===")
orders.filter(F.year("ORDER_DATE") == 2024) \
    .with_column("MONTH", F.month("ORDER_DATE")) \
    .group_by("MONTH") \
    .count() \
    .order_by("MONTH") \
    .show(12)

In [ ]:
# 注文金額の統計
print("=== 注文金額の統計 ===")
orders.select("ORDER_AMOUNT").describe().show()

---
## 1.4 チャーンの定義

### 💡 チャーンとは？

**チャーン（Churn）** とは、顧客が離反する（サービスの利用をやめる）ことです。

### 今回の定義

> **2024年前半（1-6月）に注文があったが、2024年後半（7-12月）に注文がなかった顧客**

| 条件 | 結果 |
|------|------|
| 2024年前半に1回以上注文 AND 2024年後半に0回注文 | **チャーン (1)** |
| 2024年前半に1回以上注文 AND 2024年後半に1回以上注文 | **アクティブ (0)** |

### タイムライン

| 期間 | 用途 |
|------|------|
| 2024年前半（1-6月） | 特徴量計算（この期間の行動を分析） |
| 2024年後半（7-12月） | チャーン判定（この期間に注文なし＝離反） |

In [ ]:
# 2024年前半（1-6月）に注文がある顧客
h1_customers = orders.filter(
    (F.col("ORDER_DATE") >= "2024-01-01") & 
    (F.col("ORDER_DATE") <= "2024-06-30")
).select("CUSTOMER_ID").distinct()

print(f"2024年前半に注文があった顧客数: {h1_customers.count():,}")

In [ ]:
# 2024年後半（7-12月）に注文がある顧客
h2_customers = orders.filter(
    F.col("ORDER_DATE") >= "2024-07-01"
).select("CUSTOMER_ID").distinct()

print(f"2024年後半に注文があった顧客数: {h2_customers.count():,}")

In [ ]:
# チャーン顧客 = 前半に注文あり、後半に注文なし
churned_customers = h1_customers.join(
    h2_customers, 
    on="CUSTOMER_ID", 
    how="left_anti"  # h2_customersに存在しない顧客のみ
)
churned_customers = churned_customers.with_column("IS_CHURNED", F.lit(1))

print(f"チャーン顧客数: {churned_customers.count():,}")

In [ ]:
# アクティブ顧客 = 前半に注文あり、後半にも注文あり
active_customers = h1_customers.join(
    h2_customers, 
    on="CUSTOMER_ID", 
    how="inner"  # 両方に存在する顧客
)
active_customers = active_customers.with_column("IS_CHURNED", F.lit(0))

print(f"アクティブ顧客数: {active_customers.count():,}")

In [ ]:
# チャーンラベルを結合
churn_labels = churned_customers.union_all(active_customers)

# 統計を計算
total = churn_labels.count()
churned = churn_labels.filter(F.col("IS_CHURNED") == 1).count()
churn_rate = churned / total * 100

print("=" * 50)
print("📊 チャーン統計")
print("=" * 50)
print(f"  対象顧客数（2024年前半アクティブ）: {total:,} 人")
print(f"  チャーン顧客: {churned:,} 人")
print(f"  アクティブ顧客: {total - churned:,} 人")
print(f"  チャーン率: {churn_rate:.1f}%")
print("=" * 50)

---
## 1.5 チャーンラベルの保存

In [ ]:
# チャーンラベルをテーブルに保存
churn_labels.write.save_as_table(
    "MLOPS_HOL_DB.PREP_DATA.CHURN_LABELS",
    mode="overwrite"
)

print("✅ チャーンラベルを保存しました")
print("   テーブル: MLOPS_HOL_DB.PREP_DATA.CHURN_LABELS")

In [ ]:
# 保存したテーブルを確認
saved_labels = session.table("MLOPS_HOL_DB.PREP_DATA.CHURN_LABELS")
print("=== 保存したチャーンラベル ===")
saved_labels.group_by("IS_CHURNED").count().show()

---
## 1.6 チャーン顧客の特徴分析

チャーン顧客とアクティブ顧客の違いを分析します。

In [ ]:
# 顧客情報とチャーンラベルを結合
analysis_df = churn_labels.join(
    customers,
    on="CUSTOMER_ID",
    how="inner"
)

print(f"分析対象: {analysis_df.count():,} 人")

In [ ]:
# セグメント別チャーン率
print("=== セグメント別チャーン率 ===")
analysis_df.group_by("SEGMENT").agg(
    F.count("*").alias("CUSTOMER_COUNT"),
    F.sum("IS_CHURNED").alias("CHURNED_COUNT"),
    F.round(F.sum("IS_CHURNED") * 100 / F.count("*"), 1).alias("CHURN_RATE_PCT")
).order_by("CHURN_RATE_PCT", ascending=False).show()

In [ ]:
# 地域別チャーン率
print("=== 地域別チャーン率 ===")
analysis_df.group_by("REGION").agg(
    F.count("*").alias("CUSTOMER_COUNT"),
    F.sum("IS_CHURNED").alias("CHURNED_COUNT"),
    F.round(F.sum("IS_CHURNED") * 100 / F.count("*"), 1).alias("CHURN_RATE_PCT")
).order_by("CHURN_RATE_PCT", ascending=False).show()

In [ ]:
# 2024年前半の注文金額とチャーンの関係
orders_h1 = orders.filter(
    (F.col("ORDER_DATE") >= "2024-01-01") & 
    (F.col("ORDER_DATE") <= "2024-06-30")
)

customer_h1_stats = orders_h1.group_by("CUSTOMER_ID").agg(
    F.count("*").alias("ORDER_COUNT"),
    F.sum("ORDER_AMOUNT").alias("TOTAL_AMOUNT"),
    F.avg("ORDER_AMOUNT").alias("AVG_AMOUNT")
)

analysis_with_orders = churn_labels.join(customer_h1_stats, on="CUSTOMER_ID", how="inner")

print("=== チャーン状態別の2024年前半の注文傾向 ===")
analysis_with_orders.group_by("IS_CHURNED").agg(
    F.round(F.avg("ORDER_COUNT"), 2).alias("AVG_ORDER_COUNT"),
    F.round(F.avg("TOTAL_AMOUNT"), 2).alias("AVG_TOTAL_AMOUNT"),
    F.round(F.avg("AVG_AMOUNT"), 2).alias("AVG_ORDER_AMOUNT")
).show()

---
## 1.7 発見した仮説

データ探索から、以下のような仮説が立てられます：

1. **セグメント** によってチャーン率に差がある
2. **注文回数** が少ない顧客はチャーンしやすい
3. **注文金額** が低い顧客はチャーンしやすい
4. **地域** によってチャーン率に差がある可能性

次のセクションでは、これらの仮説を検証するための特徴量をFeature Storeに登録します。

---
## ✅ Section 1 完了！

### 学んだこと
- 生データ（顧客・注文）の構造
- **チャーンの定義**（2024年前半アクティブ → 後半に注文なし）
- **ラベルの作成**（データからチャーンラベルを計算）

### 作成したもの
- **CHURN_LABELS テーブル**: 顧客ID + チャーンフラグ (IS_CHURNED)

### 次のステップ
**Section 2: Feature Store** では、特徴量を計算しFeature Storeに登録します。

➡️ `02_FEATURE_STORE` Notebookに進んでください